# Train Calc v2

This is a set of classes for expressing track configuration, and allowing easy compariosns of speed improvements.

This can be pared with speed calculations and cost estimates.

Finally this can be run through an optimizer to output optimal improvements.

This extends Alon Levy's work on [Setting Speed Zones](https://pedestrianobservations.com/2023/10/27/setting-speed-zones/)



In [1]:
class Point:
    def __init__(self, lat, long):
        self.lat, self.long = lat, long
    def distance(self, p2):
        #perform proper geo projection distance calc
        pass
    
class TrackSegment:
    """
    for initial psuedo code, I'm going to plumb in length instead of start and end points that will be wrong
    start and end points aren't super important until I add in geo functionality
    """
    def __init__(self, name=None, start=None, end=None, length=None):
        if length is None:
            assert isinstance(start, Point)
            assert isinstance(end, Point)

        #figure out how to handle this
        self.start, self.end = start, end
        self.name = name

        
    @property
    def distance(self):
        return self.start.distance(self.end)
    
    @property
    def capex(self):
        return self.capex_per_km * self.distance

    @property
    def opex(self):
        return self.opex_per_km * self.distance
        
    def calc_speed(self, operating_rules, train):
        """given the operating_rules and train specs, return the speed_limit for this segment  """
        pass
    is_electric = True
    
    def __repr__(self):
        stripped_name = str(self.__class__).replace("<class '__main__.", "").replace("'>", "")
        return "%s(%s)" % (stripped_name, self.name)

#some key points
#all track segements and trains have a capex and opex
#even if there are the existing track or rolling stock.
#this makes calculations for alternatives very easy
#it also probably allows automatic cost calcs for different track types

In [2]:
# more segment types

class StraightTrack(TrackSegment):
    # this is mostly used to capture the cost of electrifying old segments
    def __init__(self, start=None, end=None, length=None, ):
        if length is None:
            assert isinstance(start, Point)
            assert isinstance(end, Point)
        else:
            self.length = length

    def __repr__(self):
        return "Straight(class=%s, distance=%d)" % (self._class, self.length) 
    capex_per_km =  50_000 # USD
    opex_per_km  =   7_500 # USD
    is_electric = True
    _class = 2


class ClassSlowZone(TrackSegment):
    # this is mostly used to capture the cost of electrifying old segments
    def __init__(self, max_speed, start=None, end=None, length=None, ):
        if length is None:
            assert isinstance(start, Point)
            assert isinstance(end, Point)
        else:
            self.length = length
        self.max_speed = max_speed

    
    def __repr__(self):
        return "SlowZone(speed=%d, distance=%d)" % (self.max_speed, self.length) 
    capex_per_km =  50_000 # USD
    opex_per_km  =   7_500 # USD
    is_electric = True

class Class3StraightDiesel(StraightTrack):
    # this is mostly used to capture the cost of electrifying old segments
    max_speed = 60 #km/h
    capex_per_km =  50_000 # USD
    opex_per_km  =   7_500 # USD
    _class="3d"
    is_electric = False

class Class3Straight(StraightTrack):
    max_speed = 60 #km/h
    capex_per_km = 100_000 # USD
    opex_per_km  =   7_500 # USD
    _class=3

class Class4Straight(StraightTrack):
    max_speed = 100 #km/h
    capex_per_km = 150_000 # USD
    opex_per_km  =   8_500 # USD
    _class=4

class Class5Straight(StraightTrack):
    max_speed = 150 #km/h
    capex_per_km = 200_000 # USD
    opex_per_km  =  12_500 # USD
    c_lass=5

class Class6Straight(StraightTrack):
    max_speed = 200 #km/h
    capex_per_km = 275_000 # USD
    opex_per_km  =  20_500 # USD
    _class=6

In [3]:
#turns
class BasicTurn(TrackSegment):
    
    def __init__(self, start, end, radius, super_elevation):
        self.start, self.end = start, end
        self.radius, self.super_elevation = radius, super_elevation

    def distance(self):
        #distance need to be calculated differently because this is  a curve
        pass
    
    max_super_elevation = 100 # mm
    capex_per_km = 120_000 # USD
    opex_per_km  =  10_000 # USD

class MaintainedTurn(BasicTurn):
    """
    Represents a balasted turn that can take a higher super elevation, at the cost of higher opex
    this is used to represent the higher opex of greater super elevation
    """
    max_super_elevation = 150 # mm
    capex_per_km = 120_000    # USD
    opex_per_km  =  15_000    # USD


class ConcreteTurn(BasicTurn):
    """
    Represents a concrete bedded turn, expensive to build, cheap to maintain because track geometry should remain constant
    """
    max_super_elevation = 180 # mm
    capex_per_km = 220_000    # USD
    opex_per_km  =   5_000    # USD

In [4]:
#switches
class Switch(TrackSegment):
    #based on a No6 switch
    """
    it's easier to think of these as at a single point, having no length.
    for larger switches this may become a problem
    We could add a check that the distance between start and stop falls in a range
    """
    size=6
    def __init__(self):
        #no need for init parameters right now
        pass
    def __repr__(self):
        return "Switch(%d)" % self.size

    #note that we are just overriding capex and opex, not calculating them based on length
    capex =  25_000 # USD
    opex =    3_000 # USD
    max_speed =  30 #km 

class Switch8(Switch):
    size = 8
    capex =  55_000 # USD
    opex =    5_000 # USD
    max_speed =  60 #km 
    
class Switch12(Switch):
    size = 12
    capex =  250_000 # USD
    opex =    15_000 # USD
    max_speed =  130 # km 

In [5]:
class Station:
    def __init__(self, name, location):
        self.location = location #Point
        self.name = name
    low_platforms = False
    def __repr__(self):
        return "Station(%s)" % self.name

class SchedulePadding:
    #must adjoin a station
    def __init__(self, minutes=0):
        self.minutes = minutes

# Example Segments

In [6]:
GC_TO_125th = [
    Station("GrandCentral", "MidtownEast"),
    Class3Straight("throat", length=.1),
    [Switch(), Switch8(), Switch12()],
    [Switch(), Switch8(), Switch12()],
    [ClassSlowZone(max_speed=30, length=1), Class3Straight(length=1)],
    Class4Straight(length=4),
    Station("125th St", "Harlem")]
GC_TO_125th

[Station(GrandCentral),
 Straight(class=3, distance=0),
 [Switch(6), Switch(8), Switch(12)],
 [Switch(6), Switch(8), Switch(12)],
 [SlowZone(speed=30, distance=1), Straight(class=3, distance=1)],
 Straight(class=4, distance=4),
 Station(125th St)]

# Trains and operating rules

In [7]:
class TrainSet:  #Diesel
    power_weight = 15
    initial_accel = 0.2
    max_speed_km = 100

    constant_resistance  = 0.005_9 #a
    linear_resistance    = 0.000_118 # b
    quadratic_resistance = 0.00_0022 # c
    
    weight = 200_000 #not sure if this can be used for maitnenance calcs
    
    is_electric = False
    has_low_floor = False
    passengers = 800

    capex_cost = 60_000_000
    opex_per_km = 6

    mdbf = 3000  #mean distance between failures

class OperatingRules:
    """
    The set of conditions that are dictated by transit agency
    """
    def __init__(self, max_lateral_acceleration, max_super_elevation, max_cant_deficiency):
        self.max_lateral_acceleration = max_lateral_acceleration
        self.max_super_elevation = max_super_elevation
        self.max_cant_deficieny = max_cant_deficiency


# Pruning variations
Some variations are non workable (low platform trains with hgihg platform stations), some are nonsensical (specifiying expensive super elevation in excess of operating rules)
the pruning functions remove track segments from possibilities that are unworkable.  This reduces the number of track combinations that must be tried.

In [8]:
class SectionPlan:
    """Represents a concrete set of track segments, train and operating rules 
    
    Only valid sections should be insantiated no segment should be diesel required with an electric train
    """
    def __init__(self, segments, operating_rules, train):
        self.segments, self.operating_rules, self.train = segments, operating_rules, train
    
    @property
    def track_opex(self):
        return sum([t.opex for t in self.segments])
    
    @property
    def length(self):
        return sum([t.distance for t in self.segments])
    
    @property
    def section_trip_opex(self):
        """ Cost to run each train through the full section """
        return self.length * self.train.opex_per_km

    @property 
    def speed_zones(self):
        """
        return list of 
        (km_from_start, speed_limit)
        """
        #take into account max speed of train, cant deficiency
        pass
    
    @property
    def segment_times(self):
        """return list of tuples of form
            ("segment_name", "time at enter segment")
        """
        # use self.speed_zones
        #this is the best candidate for vectorization with numpy
        pass    

In [9]:
def filter_possibilities(segment_options, operating_rules, train):
    """
    simplest possible filtering,
    expand with other specific business rules
    """
    if not isinstance(segment_options, list):
        return segmen_options
    ret_options = []
    for o in segment_options:
        if train.is_electric:
            if o.is_electric:
                ret_options.append(o)
        else:
            ret_options.append(o)
    return ret_options

In [10]:
def prune_possibilities(possible_segments, operating_rules, train):
    pruned_segments = [filter_possibilities(s, operating_rules, train) for s in possible_segments]
    return pruned_segments

In [11]:
def flatten_segment_possibilities(possible_segments):
    existing_paths = [[]]
    for i, s in enumerate(possible_segments):
        if isinstance(s, list):
            new_ep = []
            for sub_s in s:
                for ep in existing_paths:
                    ab = ep.copy()
                    ab.append(sub_s)
                    new_ep.append(ab)
            existing_paths = new_ep
        else:
            [p.append(s) for p in existing_paths]
    return existing_paths

In [12]:
flt = flatten_segment_possibilities([1,2, ['a', 'b'], [True, False], 'END'])
# 1 * 1 * 2 * 2 * 1 == 4
assert(len(flt) == 4)
#each path should have 5 elements
assert(len(flt[0]) == 5)
flt

[[1, 2, 'a', True, 'END'],
 [1, 2, 'b', True, 'END'],
 [1, 2, 'a', False, 'END'],
 [1, 2, 'b', False, 'END']]

In [13]:
def produce_plans(possible_segments, rules, train):
    return [SectionPlan(segs, rules, train) for segs in flatten_segment_possibilities(possible_segments)]
    
def produce_combinations(possible_segments, possible_operating_rules, possible_trains):
    all_plans = []
    for o in possible_operating_rules:
        for t in possible_trains:
            pruned = prune_possibilities(possible_segments, o, t)
            all_plans.extend(produce_plans(pruned, o, t))
    return all_plans

In [14]:
possible_routes = flatten_segment_possibilities(GC_TO_125th)
print("1st route out of %d routes" % (len(possible_routes)))
print(possible_routes[0])


1st route out of 18 routes
[Station(GrandCentral), Straight(class=3, distance=0), Switch(6), Switch(6), SlowZone(speed=30, distance=1), Straight(class=4, distance=4), Station(125th St)]


# Example Trains

In [15]:
class TrainSet:  #Diesel
    power_weight = 15
    initial_accel = 0.2
    max_speed_km = 100

    constant_resistance  = 0.005_9 #a
    linear_resistance    = 0.000_118 # b
    quadratic_resistance = 0.00_0022 # c
    
    weight = 200_000 #not sure if this can be used for maitnenance calcs
    
    is_electric = False
    has_low_floor = False
    passengers = 800

    capex_cost = 60_000_000
    op_ex_per_km = 6

    mdbf = 3000  #mean distance between failures
    
class ElectricTrainSet:
    # TrainGeneric
    power_weight = 20
    initial_accel = 0.5
    max_speed_km = 120
    
    constant_resistance  = 0.005_9 #a
    linear_resistance    = 0.000_118 # b
    quadratic_resistance = 0.00_0022 # c
    
    weight = 200_000 #not sure if this can be used for maitnenance calcs
    
    is_electric = True
    has_low_floor = False
    passengers = 800

    capex_cost = 100_000_000
    op_ex_per_km = 4 

    mdbf = 30000  #mean distance between failures


class TrainSetN700:
    power_weight = 26.74 #k
    initial_accel = 0.9 #m
    max_speed_km = 300
    
    constant_resistance = 0.0059 #a
    linear_resistance = 0.000118 # b
    quadratic_resistance = 0.000022 # c
    
    passengers = 700
    
    is_electric = True
    has_low_floor = False
    passengers = 800
    
    capex_cost = 300_000_000
    op_ex_per_km = 6

    mdbf = 30000  #mean distance between failures
#future notes
#Schedule Padding and mdbf can be put in the optimizer to optimize for ontime %
# not sure what other types of variations could be added to deal with  schedule padding parameters
    

## Radius calcs
[Super Elevation](https://pedestrianobservations.com/2021/04/22/tilting-trains-and-technological-dead-ends/) and Cant Deficiency